In [1]:
# import numpy as np
import pandas as pd
import time
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

def sentiments(text):
    obj = SentimentIntensityAnalyzer()
    return obj.polarity_scores(text)["neg"], obj.polarity_scores(text)["pos"], obj.polarity_scores(text)["neu"], obj.polarity_scores(text)["compound"] 


def dfPrepearing(df):    
    textList = df["text"].tolist()
    newTextList = [s.replace(',', '') for s in textList]
    return newTextList


def take_date(date):
    date = str(date)
    d = date[:10]
    return d


def dfTextPrepearing(text):
    text = str(text)
    text = text.replace(',', '')
    return text


def get_dataframe_ready(df):
    date = df["date"].map(take_date)
    text_new = pd.DataFrame({"text":df["text"], "date": date})
    text_new["text"] = text_new["text"].map(dfTextPrepearing)
    text_new["date"] = pd.to_datetime(text_new["date"], format='%Y-%m-%d')
    text_new["Negative"] = 0
    text_new["Positive"] = 0
    text_new["Neutral"] = 0
    return text_new

In [2]:
#getting scraped data
text = pd.read_csv("textFrom2007.csv")
text
prepDF = get_dataframe_ready(text)
prepDF

,text,date,Negative,Positive,Neutral
0,Президент Украины Владимир Зеленский поручил п...,2020-07-30,0,0,0
1,На границе с Украиной контрабандист попытался ...,2020-07-31,0,0,0
2,Члены комитета сената США по иностранным делам...,2020-07-31,0,0,0
3,Девять задержанных в Белоруссии граждан Украин...,2020-07-31,0,0,0
4,Офис генпрокурора Украины обратился к Минску с...,2020-07-31,0,0,0
...,...,...,...,...,...
19940,Из продуктов больше всего поднялись цены на фр...,2007-01-06,0,0,0
19941,Французский актер Жерар Депардье 5 ...,2007-01-06,0,0,0
19942,Парламент Украины сегодня преодолел вето на з...,2007-01-12,0,0,0
19943,Как передает РБК-Украина инцидент повлекший см...,2007-01-17,0,0,0


In [ ]:
def setScores3(df):
    df1 = df
    text = df1['text'].to_list()
    count_neg = 0
    count_pos = 0
    count_neu = 0
    cont_string = "уеёыаоэяию"
    for i in range(len(text)):
        print(i)
        print(text[i])
        print(df1["date"].iloc[i])
        check = list(set(list(cont_string)) & set(list(text[i])))
        if len(check)==0:
            df1.iloc[i, 2] = np.nan
            df1.iloc[i, 3] = np.nan
            df1.iloc[i, 4] = np.nan
            print("Текст пропущен!!!!")
            continue
        blob = TextBlob(text[i])
        time.sleep(1)
        translated = blob.translate(to = "en")
        neg_s = 0
        pos_s = 0
        neu_s = 0
        comp_s = 0
        for sent in translated.sentences:
            neg, pos, neu, comp = sentiments(sent.raw)
            neg_s += neg
            pos_s += pos
            neu_s += neu
            comp_s += comp
        print("Рівень негативності ", neg_s)
        print("Рівень позитивності ", pos_s)
        print("Рівень нейтральності ", neu_s)
        print("COMPOSED ", comp_s)
        crit = (neg_s+pos_s)*0.4
        print("Критична межа ", crit)
        if (neg_s + pos_s) < 0.6:
            count_neu += 1
            df1.iloc[i, 4] = 1
            print("Текст є нейтральним")
            continue
        if (neg_s-pos_s) > crit:
            count_neg += 1
            df1.iloc[i, 2] = 1
            print("Текст є негативним")
        if (pos_s-neg_s) > crit:
            count_pos += 1
            df1.iloc[i, 3] = 1
            print("Текст є позитивним")
        if abs(pos_s-neg_s) <= crit:
            count_neu += 1
            df1.iloc[i, 4] = 1
            print("Текст є нейтральним")
    return df1

In [ ]:
result = setScores3(prepDF)

In [ ]:
result.to_csv("RESULT.csv", index = False)